In [3]:
"""Convert encoded csv files to one-hot-encoded npy files."""

import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm

In [1]:
def data_conversion(df,tid_col):
    """Converts input panda dataframe to one-hot-encoded Numpy array (locations are still in float)."""

    feature_size = {"lat_lon":2,"day":7,"hour":24,"category":10,"mask":1}
    
    x = [[] for i in ['lat_lon', 'day', 'hour', 'category', 'mask']]
    #x리스트안에 keys의 수 만큼 리스트가 만들어짐
    
    p_lat_centroid = (df['lat'].sum())/len(df)
    p_lon_centroid = (df['lon'].sum())/len(df)



    for tid in tqdm(df[tid_col].unique()):#df안에 tid_col의 열안의 데이터들 중에서 고유값들을 뽑아냄. ex)yellow_df의 vandorID라는 열에서 CMT랑 VTS값이 고유값이고 tid로 들어감.
        traj = df.loc[df[tid_col].isin([tid])] #traj는 DataFrame형식. transpose하기 위함
        features = np.transpose(traj.loc[:, ['lat', 'lon', 'day', 'hour', 'category']].values)
        #yellow_df가 전부 transpose돼어 열과 행이 바뀜. 그리고 맨 마지막에 .values가 있어서 결과적으로
        #pickup_longitude, pickup_latitude, drop~열 해당하는 값들이 리스트가 되어 리스트안의 리스트 형태로 feature에 저장됨
    
        p_loc_list = []
        d_loc_list = []
        
        for i in range(0, len(traj)):
            p_lat = traj['lat'].values[i] - p_lat_centroid
            p_lon = traj['lon'].values[i] - p_lon_centroid
            p_loc_list.append(np.array([p_lat, p_lon], dtype=np.float64))
                      
            
        x[0].append(p_loc_list)
        #feature안의 숫자는 df의 몇번째 컬럼인지를 의미함. eye안의 숫자는 train에서 vocab_size임
        #여기서 one hot encodding이 이루어져서 카테고리 데이터들은 0과 1의 벡터들의 배열꼴로 바뀜
        x[1].append(np.eye(7)[features[2].astype(np.int32)]) #day
        x[2].append(np.eye(24)[features[3].astype(np.int32)]) #hour
        x[3].append(np.eye(10)[features[4].astype(np.int32)]) #category
        x[4].append(np.ones(shape=(features[0].shape[0],1)))
        
    x_train = [np.array(f) for f in x]
    print(len(x_train[0]))
    

In [21]:
args = argparse.Namespace(load_path='train_latlon2.csv'
                         ,save_path='tr2.npy')

In [22]:
    df = pd.read_csv(args.load_path)

In [23]:
df

,tid,label,lat,lon,day,hour,category,tid2
0,127,6,40.834098,-73.945267,0,13,0,1
1,127,6,40.567196,-73.882576,0,19,1,1
2,127,6,40.689913,-73.981504,0,23,1,1
3,127,6,40.708588,-73.991032,0,23,1,2
4,127,6,40.833165,-73.941860,1,14,2,2
...,...,...,...,...,...,...,...,...
44803,29563,1070,40.704273,-73.986759,4,17,5,14935
44804,29563,1070,40.704733,-73.987738,4,17,6,14935
44805,29563,1070,40.695163,-73.995448,4,20,0,14936
44806,29563,1070,40.697803,-73.994145,5,8,5,14936


In [24]:
    converted_data = data_conversion(df,'tid2')
    np.save(args.save_path, converted_data)

100%|███████████████████████████████████████████████████████████████████████████| 14936/14936 [00:19<00:00, 755.61it/s]
C:\Users\USER\AppData\Local\Temp\ipykernel_17648\4101085433.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  converted_data = np.array([(f) for f in x])


ValueError: could not broadcast input array from shape (3,24) into shape (3,)

In [4]:
args = argparse.Namespace(load_path='test_latlon2.csv'
                         ,save_path='te2.npy')
df = pd.read_csv(args.load_path)
data_conversion(df,'tid2')


100%|█████████████████████████████████████████████████████████████████████████████| 7384/7384 [00:08<00:00, 831.34it/s]

7384
